> 해당 자료는 전북대학교 최규빈 교수님 2023학년도 2학기 [빅데이터분석특강](https://guebin.github.io/STBDA2023/) 자료임

# 02wk-006: 타이타닉, Autogluon (Fsize)

최규빈  
2023-09-12

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-zEjYbXMD4e-nS0_-nx9Zac&si=eA9ME6Pe4ecmtIgi>

# 2. Import

``` python
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
```

    /kaggle/input/titanic/train.csv
    /kaggle/input/titanic/test.csv
    /kaggle/input/titanic/gender_submission.csv

In [1]:
#pip install autogluon

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

# 3. 분석의 절차

## A. 데이터

`-` 비유: 문제를 받아오는 과정으로 비유할 수 있다.

In [2]:
tr = TabularDataset("~/Desktop/titanic/train.csv")
tst = TabularDataset("~/Desktop/titanic/test.csv")

`-` 피처엔지니어링

In [3]:
tr.eval('Fsize = SibSp + Parch')
tst.eval('Fsize = SibSp + Parch')

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fsize
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,2
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0


## B. Predictor 생성

`-` 비유: 문제를 풀 학생을 생성하는 과정으로 비유할 수 있다.

In [4]:
predictr = TabularPredictor("Survived")

No path specified. Models will be saved in: "AutogluonModels/ag-20230917_140239/"


## C. 적합(fit)

`-` 비유: 학생이 공부를 하는 과정으로 비유할 수 있다.

`-` 학습

In [5]:
predictr.fit(tr.eval('Fsize = SibSp + Parch'))

Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230917_140239/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   775.56 GB / 982.82 GB (78.9%)
Train Data Rows:    891
Train Data Columns: 12
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    39207.34 MB
	Train Data (Original)  M

`-` 리더보드확인 (모의고사채점)

In [6]:
predictr.leaderboard()

                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.854749       0.047879  2.622710                0.000539           0.327875            2       True         14
1         LightGBMLarge   0.832402       0.002897  0.362443                0.002897           0.362443            1       True         13
2       NeuralNetFastAI   0.832402       0.007049  0.626138                0.007049           0.626138            1       True         10
3        NeuralNetTorch   0.832402       0.008281  1.377705                0.008281           1.377705            1       True         12
4              CatBoost   0.826816       0.003585  0.415325                0.003585           0.415325            1       True          7
5               XGBoost   0.826816       0.004826  0.200229                0.004826           0.200229            1       True         11
6              LightGBM   0.821229

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.854749,0.047879,2.622710,0.000539,0.327875,2,True,14
1,LightGBMLarge,0.832402,0.002897,0.362443,0.002897,0.362443,1,True,13
2,NeuralNetFastAI,0.832402,0.007049,0.626138,0.007049,0.626138,1,True,10
3,NeuralNetTorch,0.832402,0.008281,1.377705,0.008281,1.377705,1,True,12
4,CatBoost,0.826816,0.003585,0.415325,0.003585,0.415325,1,True,7
5,XGBoost,0.826816,0.004826,0.200229,0.004826,0.200229,1,True,11
6,LightGBM,0.821229,0.002808,0.190213,0.002808,0.190213,1,True,4
7,RandomForestEntr,0.821229,0.023621,0.274035,0.023621,0.274035,1,True,6
8,LightGBMXT,0.815642,0.002782,0.195089,0.002782,0.195089,1,True,3
9,RandomForestGini,0.815642,0.024148,0.285713,0.024148,0.285713,1,True,5


## D. 예측 (predict)

`-` 비유: 학습이후에 문제를 푸는 과정으로 비유할 수 있다.

`-` training set 을 풀어봄 (predict) $\to$ 점수 확인

In [7]:
(tr.Survived == predictr.predict(tr.eval('Fsize = SibSp + Parch'))).mean()

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fd215321c10>
Traceback (most recent call last):
  File "/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


0.9438832772166106

`-` test set 을 풀어봄 (predict) $\to$ 점수 확인 하러 캐글에 결과제출

In [12]:
tst.assign(Survived = predictr.predict(tst.eval('Fsize = SibSp + Parch'))).loc[:,['PassengerId','Survived']]\
.to_csv("autogluon(Fsize)_submission.csv",index=False)